# The Kelly Rule

### Loading Libraries

In [3]:
# Pathlib
from pathlib import Path

# Numerical Computing
import numpy as np
from numpy.linalg import inv
from numpy.random import random, uniform, dirichlet, choice

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web


# Warnings
import warnings

# ZipLine Extractor
from pyfolio.utils import extract_rets_pos_txn_from_zipline

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# ScyPy
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar, newton

# Symbolic Math
from sympy import symbols, solve, log, diff

In [4]:
sns.set_style('whitegrid')

warnings.filterwarnings('ignore')

%matplotlib inline

In [5]:
np.random.seed(42)

### Loading Data

In [6]:
DATA_STORE = Path('..', 'data', 'assets.h5')

### The Optimal Size of a Bet

In [27]:
share, odds, probability = symbols('share odds probability')
Value = probability * log(1 + odds * share) + (1 - probability) * log(1 - share)
solve(diff(Value, share), share)

In [28]:
f, p = symbols('f p')
y = p * log(1 + f) + (1 - p) * log(1 - f)
solve(diff(y, f), f)

### Getting S&P 500 Data

In [29]:
with pd.HDFStore(DATA_STORE) as store:
    sp500 = store['sp500/stooq'].close

#### Compute Returns & Standard Deviation

In [30]:
annual_returns = sp500.resample('A').last().pct_change().dropna().to_frame('sp500')

In [31]:
return_params = annual_returns.sp500.rolling(25).agg(['mean', 'std']).dropna()

In [32]:
return_ci = (return_params[['mean']]
                .assign(lower=return_params['mean'].sub(return_params['std'].mul(2)))
                .assign(upper=return_params['mean'].add(return_params['std'].mul(2))))

In [33]:
return_ci.plot(lw=2, figsize=(14, 8))
plt.tight_layout()
sns.despine();

#### Kelly Rule for a Single Asset - Index Returns

In [34]:
def norm_integral(f, mean, std):
    val, er = quad(lambda s: np.log(1 + f * s) * norm.pdf(s, mean, std), 
                               mean - 3 * std, 
                               mean + 3 * std)
    return -val

In [35]:
def norm_dev_integral(f, mean, std):
    val, er = quad(lambda s: (s / (1 + f * s)) * norm.pdf(s, mean, std), m-3*std, mean+3*std)
    return val

In [36]:
def get_kelly_share(data):
    solution = minimize_scalar(norm_integral, 
                        args=(data['mean'], data['std']), 
                        bounds=[0, 2], 
                        method='bounded') 
    return solution.x

In [37]:
annual_returns['f'] = return_params.apply(get_kelly_share, axis=1)

In [38]:
return_params.plot(subplots=True, lw=2, figsize=(14, 8));

In [39]:
annual_returns.tail()

#### Performance Evaluation

In [40]:
(annual_returns[['sp500']]
 .assign(kelly=annual_returns.sp500.mul(annual_returns.f.shift()))
 .dropna()
 .loc['1900':]
 .add(1)
 .cumprod()
 .sub(1)
 .plot(lw=2));

In [43]:
annual_returns.f.describe()

In [44]:
return_ci.head()

#### Compute Kelly Fraction

In [45]:
m = .058

s = .216

In [46]:
# Option 1: Minimize the expectation integral
sol = minimize_scalar(norm_integral, args=(m, s), bounds=[0., 2.], method='bounded')
print('Optimal Kelly fraction: {:.4f}'.format(sol.x))

In [47]:
# Option 2: Take the derivative of the expectation and make it null
x0 = newton(norm_dev_integral, .1, args=(m, s))
print('Optimal Kelly fraction: {:.4f}'.format(x0))

### Kelly Rule for Multiple Assets

In [48]:
with pd.HDFStore(DATA_STORE) as store:
    sp500_stocks = store['sp500/stocks'].index 
    prices = store['quandl/wiki/prices'].adj_close.unstack('ticker').filter(sp500_stocks)

In [49]:
prices.info()

In [50]:
monthly_returns = prices.loc['1988':'2017'].resample('M').last().pct_change().dropna(how='all').dropna(axis=1)
stocks = monthly_returns.columns

monthly_returns.info()

#### Compute Precision Matrix

In [51]:
cov = monthly_returns.cov()

precision_matrix = pd.DataFrame(inv(cov), index=stocks, columns=stocks)

In [52]:
kelly_allocation = monthly_returns.mean().dot(precision_matrix)

In [53]:
kelly_allocation.describe()

In [54]:
kelly_allocation.sum()

#### Largest Portfolio Allocation

In [55]:
kelly_allocation[kelly_allocation.abs()>5].sort_values(ascending=False).plot.barh(figsize=(8, 10))
plt.yticks(fontsize=12)
sns.despine()
plt.tight_layout();

#### Performance Vs. SP500

In [56]:
ax = monthly_returns.loc['2010':].mul(kelly_allocation.div(kelly_allocation.sum())).sum(1).to_frame('Kelly').add(1).cumprod().sub(1).plot(figsize=(14,4));
sp500.filter(monthly_returns.loc['2010':].index).pct_change().add(1).cumprod().sub(1).to_frame('SP500').plot(ax=ax, legend=True)
plt.tight_layout()
sns.despine();